In [1]:
import pandas as pd
from collections import Counter

In [2]:
# load dataset
dataset = pd.read_csv('../../data/processed/29_10/wfp_food_prices_moz.csv')

In [3]:
dataset.head()

,province,district,market,latitude,longitude,category,commodity,unit,pricetype,price(MZN),year,month
0,Maputo City,Cidade_De_Maputo,Maputo,-25.965278,32.589167,cereals and tubers,Maize (white),KG,Retail,1.34,1992,11
1,Gaza,Chokwe,Chokwe,-24.533333,32.983333,cereals and tubers,Maize (white),KG,Retail,1.53,1992,12
2,Inhambane,Maxixe,Maxixe,-23.859722,35.347222,cereals and tubers,Maize (white),KG,Retail,1.69,1992,12
3,Maputo City,Cidade_De_Maputo,Maputo,-25.965278,32.589167,cereals and tubers,Maize (white),KG,Retail,1.55,1992,12
4,Gaza,Chokwe,Chokwe,-24.533333,32.983333,cereals and tubers,Maize (white),KG,Retail,1.67,1993,1


In [4]:
# check months of the year 2023
data = dataset.loc[(dataset['year'] == 2023)]

data['month'].unique()

array([1, 2, 4, 5, 6, 7], dtype=int64)

In [7]:
# remove data from years outside of 2000 to 2022
dataset = dataset.loc[(dataset['year'] >= 2000) & (dataset['year'] <= 2023)]

# remove 'non-food' category data
dataset = dataset.loc[(dataset['category'] != 'non-food')]

In [8]:
# drop features
dataset = dataset.drop(['district', 'market', 'latitude', 'longitude', 'pricetype'], axis=1)

In [9]:
# group dataset
dataset = dataset.groupby(['province', 'category', 'commodity', 'unit', 'year', 'month'])['price(MZN)'].mean().reset_index()

In [10]:
dataset.head()

,province,category,commodity,unit,year,month,price(MZN)
0,Cabo_Delgado,cereals and tubers,Cassava (dry),KG,2013,7,10.00
1,Cabo_Delgado,cereals and tubers,Cassava (dry),KG,2013,8,10.00
2,Cabo_Delgado,cereals and tubers,Cassava (dry),KG,2013,9,10.00
3,Cabo_Delgado,cereals and tubers,Cassava (dry),KG,2013,12,16.67
4,Cabo_Delgado,cereals and tubers,Cassava (fresh),KG,2013,8,20.00


In [13]:
# save grouped dataset
dataset.to_csv('../../data/processed/29_10/food_prices_grouped.csv', index=False)

In [14]:
# concatenate commodity with its unit
dataset['commodity_unit'] = dataset['commodity'] + '_' + dataset['unit']
dataset.drop(['commodity', 'unit'], axis=1, inplace=True)

dataset.rename(columns={'commodity_unit': 'commodity'}, inplace=True)

In [15]:
# count categories
columns = dataset.columns

for i in columns:
    print(f'{i}: {len(dataset[i].unique())}')

province: 11
category: 6
year: 24
month: 12
price(MZN): 9435
commodity: 53


In [17]:
# filter commodities with few occurrences
commodities = dataset['commodity'].values

max = []
min = []
k = 100

for commodity, occurrences in Counter(commodities).items():
    if occurrences >= k:
        max.append(commodity)
        print(f'{commodity}: {occurrences}')
        
    else:
        min.append(commodity)
        #print(f'{commodity}: {occurrences}')
        
    # print(f'{commodity}: {occurrences}')

print()
print(f'total commodities with more than {k} occurrences: {len(max)}')
print(f'total commodities with less than {k} occurrences: {len(min)}')

Cassava (dry)_KG: 197
Maize (white)_KG: 2433
Maize meal_25 KG: 213
Maize meal (white, first grade)_KG: 1266
Maize meal (white, with bran)_KG: 700
Maize meal (white, without bran)_KG: 783
Potatoes_KG: 224
Rice_25 KG: 225
Rice_KG: 1854
Rice (imported)_KG: 2049
Rice (local)_KG: 496
Sweet potatoes_KG: 253
Wheat flour (local)_KG: 1274
Eggs_30 pcs: 222
Fish_500 G: 206
Salt (iodised)_KG: 222
Sugar_KG: 222
Sugar (brown, imported)_KG: 145
Sugar (brown, local)_KG: 1984
Oil (vegetable)_5 L: 223
Oil (vegetable, imported)_L: 291
Oil (vegetable, local)_L: 1994
Beans (butter)_KG: 848
Beans (catarino)_KG: 788
Beans (dry)_KG: 1178
Beans (magnum)_KG: 489
Cowpeas_KG: 1316
Groundnuts_KG: 218
Groundnuts (Mix)_KG: 545
Groundnuts (large, shelled)_KG: 1090
Groundnuts (small, shelled)_KG: 1128
Cabbage_KG: 253
Carrots_KG: 250
Cassava leaves_KG: 214
Coconut_Unit: 223
Garlic_KG: 223
Kale_KG: 220
Onions_KG: 257
Tomatoes_KG: 260
Cassava flour_KG: 166

total commodities with more than 100 occurrences: 40
total commo

In [18]:
# remove commodities with few occurrences
dataset = dataset.loc[dataset['commodity'].isin(max)]

In [19]:
# convert categorical features with one hot encoding
features = ['province',	'category',	'commodity']

dataset_encoded = dataset
    
for i in features:
    dataset_encoded = pd.get_dummies(dataset_encoded, columns=[i], prefix=[i])

In [20]:
dataset_encoded.head()

,year,month,price(MZN),province_Cabo_Delgado,province_Gaza,province_Inhambane,province_Manica,province_Maputo,province_Maputo City,province_Nampula,...,commodity_Rice (local)_KG,commodity_Rice_25 KG,commodity_Rice_KG,commodity_Salt (iodised)_KG,"commodity_Sugar (brown, imported)_KG","commodity_Sugar (brown, local)_KG",commodity_Sugar_KG,commodity_Sweet potatoes_KG,commodity_Tomatoes_KG,commodity_Wheat flour (local)_KG
0,2013,7,10.00,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2013,8,10.00,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2013,9,10.00,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2013,12,16.67,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,2000,2,1.14,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
# save encoded dataset
dataset_encoded.to_csv('../../data/processed/29_10/food_prices_encoded.csv', index=False)